In [1]:
import os
import boto3
import pandas as pd

In [2]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Bucket name and file paths
bucket_name = 'ca-climate-index'
directory = '0_map_data/crosswalk_data/CCI_Projects_Project_Category_Update_02142024.xlsm'

print('Pulling file')
s3_client.download_file(bucket_name, directory, 'CCI_Projects_Project_Category_Update_02142024.xlsm')
print('File pulled')

Pulling file
File pulled


In [3]:
crosswalk_data = pd.read_excel('CCI_Projects_Project_Category_Update_02142024.xlsm')

## How many rows within the original dataset?

In [4]:
pd.set_option('display.max_columns', None)
print('Number of rows within dataset:', len(crosswalk_data))
display(crosswalk_data)


Number of rows within dataset: 133698


,Project IDNumber,Reporting Cycle Name,Agency Name,Program Name,Program Description,Sub Program Name,Record Type,Project Name,Project Type,Project Description,SECTOR,CATEGORY,ACTION,Census Tract,Address,Lat Long,Senate\nDistrict,Assembly\nDistrict,County,Total Project Cost,Total Program GGRFFunding,Project Life Years,Total Project GHGReductions,Annual Project GHGReductions,Project Count,Fiscal Year Funding Project,Is Benefit Disadvantaged Communities,Disadvantaged Community Criteria,Disadvantaged Community Need,Disadvantaged Community Census Tracts,Total GGRFDisadvantaged Community Funding,Disadvantaged Community Benefits Description,Funding Benefiting Disadvantaged Communities,Estimated Num Vehicles In Service,Funding Within Disadvantage Communities,Other Project Benefits Description,VMTReductions,Number Of Housing Units,Number Of Affordable Housing Units,Estimated Number Of Trees To Be Planted,Energy Cost Savings,Estimated Energy Saved KWH,Estimated Energy Saved Therms,Estimated Water Saved Gallons,Estimated Energy Generated KWH,Estimated Fuel Use Reduction Gal,Vouchers Benefiting Disadvantaged Communities,Number Of Rebates Issued,Rebates Within Disadvantaged Communities,Date Operational,Project Completion Date,Date Imported,Funding Recipient,AB1550Choice,Buffer Amount,Buffer Count,Is AB1550Buffer Region,CESVersion,CESVersion Calc,DAC1550Amount,DAC1550Count,Is Benefit DAC1550Communities,DACTable,FG17Comm Need,FG17Comm Need Qual,DACCommunity Benefit Critieria Met,Date Selected For Award,Low Income Amount,Low Income Count,Low Income Household Amount,Low Income Household Count,Is Low Income Communities,Potential Buffer Amount,Potential Buffer Count,Potential DAC1550Amount,Potential DAC1550Count,Potential Low Income Amount,Potential Low Income Count,Estimated Acres Preserved,Estimated Acres Restored,Estimated Acres Treated,Estimated Diverted From Landfills Tons,Renewable Fuel Generation Tons,Wood Burning Reduction Cords,Estimated Ridership Increases,Black Carbon Reductions Pounds,StateEW_DPM,StateEW_Nox,StateEW_PM25,StateEW_ROG,Diesel Pm Reductions Pounds,NOx Reductions Pounds,Pm25Reductions Pounds,Rog Reductions Pounds,Estimated Total Recycling Tons,Estimated Waste Digested Tons,Reclaimed Food Tons,NUMBER OF PLANS,POLLINATOR ACREAGE,RESEARCH GRANT,SCIENCE ADVANCEMENT,SOIL BENEFIT,TRAVEL COST SAVINGS,EST ENERGY GEN SCF,EST SOURCE RED TONS,FUEL TREATMENT NUM,EDUCATION EVENT NUM,ENERGY AUDIT BUILDINGS,EST DIVERT LANDFILLS TONS,EST FUEL GEN GAL,Voucher ID,Voucher Name,Voucher Description,Direct Jobs Fte,Indirect Jobs Fte,Induced Jobs Fte,Climate Adaptation,Community Engagement,Compost Produced Tons,Compost Produced Tons Yr,Net Density DUA,Applicants Assisted,Invasive Cover 12 Months,Invasive Cover 36 Months,Project Acreage,IS IAE,Intermediary Admin Expenses Calc,PRIMARY_FUNDING_RECIPIENT_TYPE,TRIBAL AFFILIATION,PROJECT PARTNERS
0,G13-AQIP-01,2015,California Air Resources Board,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,IMPLEMENT,NaN,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,NaN,NaN,07,14,Alameda,14833.0,14833.0,3,50.0,0.0,7.0,NaN,0.0,NaN,NaN,NaN,0.0,Reduction of Criteria Air Pollutant or Toxic A...,0.0,0.0,0.0,CVRP promotes clean vehicle adoption in Califo...,0.0,0.0,0.0,0.0,9455.0,0.0,0.0,0.0,0.0,5110.0,0.0,7.0,0.0,2013-09-21,01/05/2015,2023-06-05 09:18:14.087,Center for Sustainable Energy (CSE),NaN,NaN,NaN,0.0,2.0,535.0,NaN,NaN,0.0,Sustainable Transportation,D,Common Need,A,NaT,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,CVRP,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,G13-AQIP-01,2015,California Air Resources Board,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clea

## Display all columns

In [5]:
print('Number of columns:', len(crosswalk_data.columns.tolist()))
display(crosswalk_data.columns.tolist())

Number of columns: 130


['Project IDNumber',
 'Reporting Cycle Name',
 'Agency Name',
 'Program Name',
 'Program Description',
 'Sub Program Name',
 'Record Type',
 'Project Name',
 'Project Type',
 'Project Description',
 'SECTOR',
 'CATEGORY',
 'ACTION',
 'Census Tract',
 'Address',
 'Lat Long',
 'Senate\nDistrict',
 'Assembly\nDistrict',
 'County',
 'Total Project Cost',
 'Total Program GGRFFunding',
 'Project Life Years',
 'Total Project GHGReductions',
 'Annual Project GHGReductions',
 'Project Count',
 'Fiscal Year Funding Project',
 'Is Benefit Disadvantaged Communities',
 'Disadvantaged Community Criteria',
 'Disadvantaged Community Need',
 'Disadvantaged Community Census Tracts',
 'Total GGRFDisadvantaged Community Funding',
 'Disadvantaged Community Benefits Description',
 'Funding Benefiting Disadvantaged Communities',
 'Estimated Num Vehicles In Service',
 'Funding Within Disadvantage Communities',
 'Other Project Benefits Description',
 'VMTReductions',
 'Number Of Housing Units',
 'Number Of Aff

## Selecting columns relevant to initial analysis

In [6]:
relevant_columns = [
    'Program Name',
    'Program Description',
    'Sub Program Name',
    'Project Type',
    'Project Description',
    'SECTOR',
    'CATEGORY',
    'ACTION',
    'Census Tract',
    'Total Project GHGReductions',
    'Project Count'
]

In [7]:
data_of_interest = crosswalk_data[relevant_columns]

In [8]:
# Set display options to show all columns and rows
# pd.set_option('display.max_columns', None)  # To display all columns
# pd.set_option('display.max_rows', None)     # To display all rows

# Now display data_of_interest
display(data_of_interest)

,Program Name,Program Description,Sub Program Name,Project Type,Project Description,SECTOR,CATEGORY,ACTION,Census Tract,Total Project GHGReductions,Project Count
0,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,50.0,7.0
1,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,41.0,7.0
2,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,97.0,14.0
3,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,68.0,10.0
4,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,29.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
133693,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0
133694,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC improves expertise on climate change issu...,Climate Adaptation and Resilience,Sea Level Rise,Capacity-Building,NaN,0.0,1.0
133695,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0
133696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Rollover climate risk keywords subset from v1 crosswalk

In [172]:
climate_risk_dict = {
    'wildfire mitigation': ['wildfire', 'prescribed fire', 'fire prevention', 'burn', 'controlled burn', 'controlled_burning', 'prescribed burn', 'prescribed burning' 'firefighting', 'reforest', 'reforestation', 'vegetation management', 'roadside brushing', 'fuel break'],
    'sea level rise mitigation': ['sea level rise', 'slr', 'seawall', 'riparian', 'seawalls', 'shoreline', 'wetland', 'mangrove'],
    'extreme heat mitigation': ['extreme heat', 'shade', 'shading', 'cooling center', 'cooling centers', 'heat-resistant', 'heat resistant', 'heat reducing', 'heat-reducing'],
    'drought mitigation': ['drought', 'irrigation', 'soil moisture', 'rainwater harvest', 'rainwater harvesting', 'water storage', 'water allocation', 'water management'],
    'inland flooding mitigation': ['flooding', 'runoff', 'inland flood', 'inland flooding', 'floodplain', 'floodproof', 'floodproofing', 'elevated flood', 'flood barrier', 'flood barriers', 'drainage'],
    'greenhouse gas mitigation': ['ghg', 'GHG', 'greenhouse gas', 'emission', 'emissions', 'carbon sequestration', 'electrification', 'carbon capture', 'solar power', 'wind energy', 'hydroelectricity', 'geothermal energy', 'biomass energy', 'Energy-efficiency']
}


## How many unique entries for each column? Will help decide which column to start with

In [178]:
def count_entries(dataframe):
    total_entries = dataframe.count()
    unique_entries = dataframe.nunique()
    return total_entries, unique_entries

# Example usage:
total_entries, unique_entries = count_entries(data_of_interest)
print("Total entries per column:")
print(total_entries)
print("\nUnique entries per column:")
print(unique_entries)


Total entries per column:
Program Name                   133696
Program Description            133694
Sub Program Name               133696
Project Type                    19903
Project Description            133698
SECTOR                         133697
CATEGORY                       133697
ACTION                         133697
Census Tract                   114823
Total Project GHGReductions    133696
Project Count                  133696
dtype: int64

Unique entries per column:
Program Name                     39
Program Description              19
Sub Program Name                 76
Project Type                    963
Project Description            5236
SECTOR                           25
CATEGORY                         77
ACTION                           95
Census Tract                   7978
Total Project GHGReductions    2604
Project Count                   243
dtype: int64


## Deciding to loop through 'Project Description' first as it has a large number of variation to capture many datasets, and makes most practical scense for filtering climate keywords

In [173]:
def add_climate_risk_column(df, keyword_dict):
    # Initialize new columns to store climate risk mitigation keywords, detected values, repeat counts, and total unique descriptions
    df['Climate_Risk_Mitigation'] = ''
    df['Detected_Climate_Risk_Mitigation_Keyword'] = ''
    df['Repeat_Project_Description_Count'] = 0

    # Initialize a counter for each keyword
    keyword_counter = {keyword: 0 for keyword in keyword_dict}

    # Create a dictionary to store the repeat count for each unique project description
    description_counts = {}

    # Iterate through each unique description
    unique_descriptions = df['Project Description'].unique()
    total_unique_descriptions = len(unique_descriptions)

    for description in unique_descriptions:
        # Find all rows with this description
        description_rows = df[df['Project Description'] == description]
        repeat_count = len(description_rows)
        # Update the repeat count for this description
        description_counts[description] = repeat_count

        # Iterate through each row with this description
        for index, row in description_rows.iterrows():
            keywords_found = set()  # To store unique keywords found in each row
            detected_values = []    # To store the detected values for each row
            # Iterate through each keyword in the dictionary
            for keyword, values in keyword_dict.items():
                # Check if any value of the keyword is present in the description (case-insensitive)
                detected = [val for val in values if val.lower() in description.lower()]
                if detected:
                    keywords_found.add(keyword)
                    keyword_counter[keyword] += 1
                    detected_values.extend(detected)

            # Update the 'Climate_Risk_Mitigation' column with unique keywords found
            df.at[index, 'Climate_Risk_Mitigation'] = ', '.join(keywords_found)
            # Update the 'Detected_Values' column with detected values
            df.at[index, 'Detected_Climate_Risk_Mitigation_Keyword'] = ', '.join(detected_values)
            # Update the 'Repeat_Project_Description_Count' column with the repeat count for this description
            df.at[index, 'Repeat_Project_Description_Count'] = repeat_count

    # Print keyword counts
    print("Keyword Counts:")
    for keyword, count in keyword_counter.items():
        print(f"{keyword}: {count}")
    print('')
    # Print total unique descriptions count
    print(f"Total Unique Descriptions: {total_unique_descriptions}")
    print('')

    # Print unique counts for each keyword
    print("Unique Counts for Each Keyword:")
    for keyword, values in keyword_dict.items():
        unique_count = sum(1 for description in unique_descriptions if any(val.lower() in description.lower() for val in values))
        print(f"{keyword}: {unique_count}")


In [177]:
add_climate_risk_column(crosswalk_data, climate_risk_dict)
pd.set_option('display.max_colwidth', None)
data_preview = crosswalk_data[['Project Description', 'Repeat_Project_Description_Count', 'Detected_Climate_Risk_Mitigation_Keyword', 'Climate_Risk_Mitigation']]

# Filter the DataFrame to show only rows with entries in the 'Climate_Risk_Mitigation' column
data_preview_filtered = data_preview[data_preview['Climate_Risk_Mitigation'] != '']

# Filter the DataFrame to show only rows with entries in the 'Climate_Risk_Mitigation' column
data_preview_filtered = data_preview[data_preview['Climate_Risk_Mitigation'] != '']

# Sort the DataFrame based on 'Repeat_Project_Description_Count' in descending order
data_preview_filtered_sorted = data_preview_filtered.sort_values(by='Repeat_Project_Description_Count', ascending=False)

# Drop duplicates based on both 'Repeat_Project_Description_Count' and 'Project Description' to keep only one row per unique combination
data_preview_filtered_unique = data_preview_filtered_sorted.drop_duplicates(subset=['Repeat_Project_Description_Count', 'Project Description'])

#display(data_preview_filtered_unique)
display(data_preview_filtered_unique[:40])

Keyword Counts:
wildfire mitigation: 1261
sea level rise mitigation: 111
extreme heat mitigation: 186
drought mitigation: 1143
inland flooding mitigation: 133
greenhouse gas mitigation: 106853

Total Unique Descriptions: 5236

Unique Counts for Each Keyword:
wildfire mitigation: 695
sea level rise mitigation: 80
extreme heat mitigation: 130
drought mitigation: 650
inland flooding mitigation: 59
greenhouse gas mitigation: 846


,Project Description,Repeat_Project_Description_Count,Detected_Climate_Risk_Mitigation_Keyword,Climate_Risk_Mitigation
0,"CVRP promotes clean vehicle adoption in California by offering rebates from $1,000 to $7,502 for the purchase or lease of new, eligible zero-emission vehicles, including electric, plug-in hybrid electric and fuel cell vehicles.",82945,emission,greenhouse gas mitigation
91768,"Reduces agricultural sector emissions by providing incentives for agricultural harvesting equipment, heavy-duty trucks, agricultural pump engines, tractors, and other equipment used in agricultural operations",7787,"emission, emissions",greenhouse gas mitigation
111151,"Enhanced Fleet Modernization Program Plus-up Pilot Project (EFMP Plus-up) operates in conjunction with EFMP, the voluntary car scrap and replacement program implemented by ARB and local air districts in coordination with the Bureau of Automotive Repair. EFMP Plus-up provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who retire older vehicles and replace them with cleaner used or new hybrid, plug-in hybrid, or zero-emission vehicles. Funded with auction proceeds in 2014-15, 2015-16, and 2016-17. 2014‑15 and 2015-16 funding was limited to the San Joaquin Valley and South Coast air districts; 2016-17 funding is available to any air district that implements a vehicle scrap and replacement program that meets the minimum requirements established in ARB’s EFMP regulation and requests to participate.",6191,emission,greenhouse gas mitigation
42465,"Funding as per CAP Incentives Program, Carl Moyer Program and Proposition 1B Program to reduce air pollution and greenhouse gas emissions",3587,"greenhouse gas, emission, emissions",greenhouse gas mitigation
126858,"Clean Cars 4 All operates in conjunction with EFMP, the voluntary car scrap and replacement program implemented by CARB and local air districts in coordination with the Bureau of Automotive Repair. CC4A provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who retire older vehicles and replace them with cleaner used or new hybrid, plug-in hybrid, or zero-emission vehicles. Funded with auction proceeds in 2014-15, 2015-16, and 2016-17. 2014‑15 and 2015-16 funding was limited to the San Joaquin Valley and South Coast air districts; 2021-2022 funding was available to any air district that implements a vehicle scrap and replacement program that meets the minimum requirements established in CARB's CC4A regulation and requests to participate.",1816,emission,greenhouse gas mitigation
22496,EFMP Plus-up provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who replace older vehicles with cleaner used or new low/zero emission vehicles.,1308,emission,greenhouse gas mitigation
121314,"Provides combination of a vehicle price buy-down with a low-interest loan to lower-income consumers to purchase a new or used hybrid, plug-in hybrid, or zero-emission vehicle. Plug-in hybrid and zero-emission vehicle buyers can receive an EVSE incentive. Financial and vehicle education are provided for consumer protection.",1136,emission,greenhouse gas mitigation
115133,"$5 million grant to offer low-income residents low cost loans (up to 8 percent APR) and vehicle cost buy-down grants (up to $5,000) for the purchase of used or new clean vehicles (hybrid, plug-in hybrid, battery and fuel-cell zero-emission).",529,emission,greenhouse gas mitigation
61363,"Provide incentive rebates for single-family homeowners in Fresno, Clovis, and Reedley (Fresno County) to replace turf grass with Central Valley-appropriate drought tolerant landscapes.",261,drought,drought mitigation
77866,"Funds cleaner school buses including zero-emission school buses, near-z

## To view randomized ones within the dataset just for further keyword filtering

In [176]:
import random

crosswalk_data_lite = crosswalk_data.sample(n=1000)

add_climate_risk_column(crosswalk_data_lite, climate_risk_dict)
pd.set_option('display.max_colwidth', None)
data_preview = crosswalk_data_lite[['Project Description', 'Repeat_Project_Description_Count', 'Detected_Climate_Risk_Mitigation_Keyword', 'Climate_Risk_Mitigation']]

# Filter the DataFrame to show only rows with entries in the 'Climate_Risk_Mitigation' column
data_preview_filtered = data_preview[data_preview['Climate_Risk_Mitigation'] != '']

# Sort the DataFrame based on 'Repeat_Project_Description_Count' in descending order
data_preview_filtered_sorted = data_preview_filtered.sort_values(by='Repeat_Project_Description_Count', ascending=False)

# Drop duplicates based on both 'Repeat_Project_Description_Count' and 'Project Description' to keep only one row per unique combination
data_preview_filtered_unique = data_preview_filtered_sorted.drop_duplicates(subset=['Repeat_Project_Description_Count', 'Project Description'])

#display(data_preview_filtered_unique)
display(data_preview_filtered_unique[:25])

Keyword Counts:
wildfire mitigation: 7
sea level rise mitigation: 2
extreme heat mitigation: 1
drought mitigation: 8
inland flooding mitigation: 0
greenhouse gas mitigation: 800

Total Unique Descriptions: 89

Unique Counts for Each Keyword:
wildfire mitigation: 7
sea level rise mitigation: 2
extreme heat mitigation: 1
drought mitigation: 8
inland flooding mitigation: 0
greenhouse gas mitigation: 15


,Project Description,Repeat_Project_Description_Count,Detected_Climate_Risk_Mitigation_Keyword,Climate_Risk_Mitigation
98880,"CVRP promotes clean vehicle adoption in California by offering rebates from $1,000 to $7,502 for the purchase or lease of new, eligible zero-emission vehicles, including electric, plug-in hybrid electric and fuel cell vehicles.",633,emission,greenhouse gas mitigation
62865,"Reduces agricultural sector emissions by providing incentives for agricultural harvesting equipment, heavy-duty trucks, agricultural pump engines, tractors, and other equipment used in agricultural operations",65,"emission, emissions",greenhouse gas mitigation
42755,"Funding as per CAP Incentives Program, Carl Moyer Program and Proposition 1B Program to reduce air pollution and greenhouse gas emissions",32,"greenhouse gas, emission, emissions",greenhouse gas mitigation
117935,"Enhanced Fleet Modernization Program Plus-up Pilot Project (EFMP Plus-up) operates in conjunction with EFMP, the voluntary car scrap and replacement program implemented by ARB and local air districts in coordination with the Bureau of Automotive Repair. EFMP Plus-up provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who retire older vehicles and replace them with cleaner used or new hybrid, plug-in hybrid, or zero-emission vehicles. Funded with auction proceeds in 2014-15, 2015-16, and 2016-17. 2014‑15 and 2015-16 funding was limited to the San Joaquin Valley and South Coast air districts; 2016-17 funding is available to any air district that implements a vehicle scrap and replacement program that meets the minimum requirements established in ARB’s EFMP regulation and requests to participate.",29,emission,greenhouse gas mitigation
22429,EFMP Plus-up provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who replace older vehicles with cleaner used or new low/zero emission vehicles.,12,emission,greenhouse gas mitigation
127060,"Clean Cars 4 All operates in conjunction with EFMP, the voluntary car scrap and replacement program implemented by CARB and local air districts in coordination with the Bureau of Automotive Repair. CC4A provides additional incentives above the base EFMP incentive for lower-income consumers living in ZIP codes containing a disadvantaged community census tract who retire older vehicles and replace them with cleaner used or new hybrid, plug-in hybrid, or zero-emission vehicles. Funded with auction proceeds in 2014-15, 2015-16, and 2016-17. 2014‑15 and 2015-16 funding was limited to the San Joaquin Valley and South Coast air districts; 2021-2022 funding was available to any air district that implements a vehicle scrap and replacement program that meets the minimum requirements established in CARB's CC4A regulation and requests to participate.",9,emission,greenhouse gas mitigation
121898,"Provides combination of a vehicle price buy-down with a low-interest loan to lower-income consumers to purchase a new or used hybrid, plug-in hybrid, or zero-emission vehicle. Plug-in hybrid and zero-emission vehicle buyers can receive an EVSE incentive. Financial and vehicle education are provided for consumer protection.",7,emission,greenhouse gas mitigation
77857,"Funds cleaner school buses including zero-emission school buses, near-zero-emission plug-in hybrid school buses, and school buses with internal combustion engines or hybrid school buses operating on renewable fuels that reduce GHG emissions. Priority is given to funding buses used in small- and medium-size air districts.",4,"ghg, GHG, emission, emissions",greenhouse gas mitigation
47538,"$5 million grant to offer low-income residents low cost loans (up to 8 percent APR) and vehicle cost buy-down grants (up to $5,000) for the purchase of used or new clean vehicles (hybrid, plug-in hybrid, ba